In [ ]:
import kagglehub
import torch
import torch.optim as optim
from data_loader_BERT import DataLoader4BERT
from model import TransformerForNumericPrediction
from utils import train, evaluate, adjust_learning_rate
from config import LEARNING_RATE, NUM_EPOCHS, PREVIOUS_LOSS, BERT_MODEL_NAME

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
path = kagglehub.dataset_download("samiraalipour/genomics-of-drug-sensitivity-in-cancer-gdsc")
print("Path to dataset files:", path)

dataloader = DataLoader4BERT(
    gdsc_path = path + '/GDSC_DATASET.csv',
    compounds_path = path + '/Compounds-annotation.csv',
    gdsc2_path = path + '/GDSC2-dataset.csv',
    cell_lines_path = path + '/Cell_Lines_Details.xlsx',
)

X_train_numeric, X_train_text, y_train_tensor, X_test_numeric, X_test_text, y_test_tensor = dataloader.get_data()

# Move necessary data to GPU
input_ids_train = X_train_text["input_ids"].to(device)
attention_mask_train = X_train_text["attention_mask"].to(device)
X_train_numeric = X_train_numeric.to(device)
y_train_tensor = y_train_tensor.to(device)

input_ids_test = X_test_text["input_ids"].to(device)
attention_mask_test = X_test_text["attention_mask"].to(device)
X_test_numeric = X_test_numeric.to(device)
y_test_tensor = y_test_tensor.to(device)

In [2]:
# Load pre-trained model
model = TransformerForNumericPrediction(model_name=BERT_MODEL_NAME, num_numeric_features=X_train_numeric.shape[1])
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Training loop
num_epochs = NUM_EPOCHS
previous_loss = PREVIOUS_LOSS
for epoch in range(num_epochs):
    train_loss = train(model, input_ids_train, attention_mask_train, X_train_numeric, y_train_tensor, optimizer)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}")
    
    # Adjust learning rate based on loss
    adjust_learning_rate(optimizer, train_loss, previous_loss)
    previous_loss = train_loss

Epoch 1/20, Training Loss: 4.5950
Epoch 2/20, Training Loss: 3.1991
Epoch 3/20, Training Loss: 3.0998
Epoch 4/20, Training Loss: 3.1003
Learning rate decreased to 1.800000e-05
Epoch 5/20, Training Loss: 3.0986
Epoch 6/20, Training Loss: 3.0981
Epoch 7/20, Training Loss: 3.0980
Epoch 8/20, Training Loss: 3.0974
Epoch 9/20, Training Loss: 3.0971
Epoch 10/20, Training Loss: 3.0968
Epoch 11/20, Training Loss: 3.0966
Epoch 12/20, Training Loss: 3.0969
Learning rate decreased to 1.620000e-05
Epoch 13/20, Training Loss: 3.0961
Epoch 14/20, Training Loss: 3.0962
Learning rate decreased to 1.458000e-05
Epoch 15/20, Training Loss: 3.0958
Epoch 16/20, Training Loss: 3.0961
Learning rate decreased to 1.312200e-05
Epoch 17/20, Training Loss: 3.0956
Epoch 18/20, Training Loss: 3.0950
Epoch 19/20, Training Loss: 3.0950
Learning rate decreased to 1.180980e-05
Epoch 20/20, Training Loss: 3.0948


In [4]:
# Evaluate on test data
mae, mse = evaluate(model, input_ids_test, attention_mask_test, X_test_numeric, y_test_tensor)
# print(f"Mean Absolute Error on Test Data: {mae:.4f}")
print(f"Mean Square Error on Test Data: {mse:.4f}")

CSV file 'BERT_results.csv' has been created!
Mean Square Error on Test Data: 3.0844


## Epoch 5

roberta Mean Square Error on Test Data: 3.1066

microsoft/deberta-v3-base Mean Square Error on Test Data: 3.1144

## Epoch 20

bert-base-uncased Mean Square Error on Test Data: 3.0704

roberta-base Mean Square Error on Test Data: 3.0844

microsoft/deberta-v3-base Mean Square Error on Test Data: 3.0943

microsoft/deberta-v3-large Mean Square Error on Test Data: 3.1113